In [235]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from subprocess import check_output

Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

Step 1: Load data

In [236]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [237]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [238]:
df_test = pd.read_csv('../input/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [239]:
df_test.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Step 2: Basic cleaning and preprocessing

In [240]:
# label encode sex
le = LabelEncoder()
le.fit(df_train.Sex)
df_train.loc[:,'Sex'] = le.transform(df_train.Sex)
df_test.loc[:,'Sex'] = le.transform(df_test.Sex)


In [241]:
# Fill in the missing values
mAge = pd.concat((df_train.Age, df_test.Age), axis=0).mean()
df_train.loc[:, 'Age'] = df_train.Age.fillna(mAge)
df_test.loc[:, 'Age'] = df_test.Age.fillna(mAge)

# Step 3: Feature engineering

In [242]:
df_train['Age_binned'] = pd.cut(df_train['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)
df_test['Age_binned'] = pd.cut(df_test['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)

In [243]:
df_test['Embarked'].fillna('S',inplace=True)
df_train['Embarked'].fillna('S',inplace=True)

In [244]:
df_train['Family_size'] = df_train['SibSp'] + df_train['Parch']
df_test['Family_size'] = df_test['SibSp'] + df_test['Parch']

In [245]:
df_train['Is_Alone'] = (df_train['Family_size'] == 0).astype(int)
df_train['Is_Alone'].drop

df_test['Is_Alone'] = (df_test['Family_size'] == 0).astype(int)
df_test['Is_Alone'].drop

<bound method Series.drop of 0      1
1      0
2      1
3      1
4      0
5      1
6      1
7      0
8      1
9      0
10     1
11     1
12     0
13     0
14     0
15     0
16     1
17     1
18     0
19     1
20     0
21     0
22     1
23     0
24     0
25     0
26     0
27     1
28     1
29     0
      ..
388    1
389    0
390    1
391    0
392    0
393    1
394    0
395    0
396    1
397    0
398    1
399    1
400    1
401    0
402    0
403    1
404    0
405    1
406    0
407    0
408    1
409    0
410    1
411    0
412    1
413    1
414    1
415    1
416    1
417    0
Name: Is_Alone, Length: 418, dtype: int64>

In [246]:
le.fit(df_train.Embarked)
df_train.loc[:,'Embarked'] = le.transform(df_train.Embarked)
df_test.loc[:,'Embarked'] = le.transform(df_test.Embarked)

In [247]:
df_train = df_train.drop(columns = ['Name','Age','SibSp','Parch','Ticket','Fare','Cabin'], axis=1, inplace = False)
df_test = df_test.drop(columns = ['Name','Age','SibSp','Parch','Ticket','Fare','Cabin'], axis=1, inplace = False)

In [248]:
df_train

,PassengerId,Survived,Pclass,Sex,Embarked,Age_binned,Family_size,Is_Alone
0,1,0,3,1,2,1,1,0
1,2,1,1,0,0,2,1,0
2,3,1,3,0,2,1,0,1
3,4,1,1,0,2,2,1,0
4,5,0,3,1,2,2,0,1
5,6,0,3,1,1,1,0,1
6,7,0,1,1,2,3,0,1
7,8,0,3,1,2,0,4,0
8,9,1,3,0,2,1,2,0
9,10,1,2,0,0,0,1,0


In [249]:


df_test


,PassengerId,Pclass,Sex,Embarked,Age_binned,Family_size,Is_Alone
0,892,3,1,1,2,0,1
1,893,3,0,2,2,1,0
2,894,2,1,1,3,0,1
3,895,3,1,2,1,0,1
4,896,3,0,2,1,2,0
5,897,3,1,2,0,0,1
6,898,3,0,1,1,0,1
7,899,2,1,2,1,2,0
8,900,3,0,0,1,0,1
9,901,3,1,2,1,2,0


## Step 4: Format data for modelling

In [250]:
# Split data into X and y, and select features to use
X = df_train.loc[:,['Pclass','Sex','Embarked','Age_binned','Family_size','Is_Alone']]
y = df_train.loc[:,'Survived']

In [251]:
enc = OneHotEncoder()

enc.fit(X)

Xe = enc.transform(X).toarray()
Xe.shape

(891, 24)

In [252]:
Xe
type(Xe)


numpy.ndarray

In [253]:
#Xcat = X['Pclass','Sex','Embarked','Age_binned','Family_size','Is_Alone']

#enc = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

#enc.fit_transform(Xcat)

#myEncoder = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
#myEncoder.fit(X)
#X = pd.concat(['Pclass','Sex','Embarked','Age_binned','Family_size','Is_Alone'])

   #X,
    #pd.DataFrame(myEncoder.transform(X.toarray#X['Pclass','Sex','Embarked','Age_binned','Is_Alone'])).toarray()
     #], axis = 1).reindex_axis(x.columns, axis = 1)
#myEncoder = OneHotEncoder(categorical_features = X.drop(columns = 'Survived'))

#X = myEncoder.fit(

In [254]:
X

,Pclass,Sex,Embarked,Age_binned,Family_size,Is_Alone
0,3,1,2,1,1,0
1,1,0,0,2,1,0
2,3,0,2,1,0,1
3,1,0,2,2,1,0
4,3,1,2,2,0,1
5,3,1,1,1,0,1
6,1,1,2,3,0,1
7,3,1,2,0,4,0
8,3,0,2,1,2,0
9,2,0,0,0,1,0


In [257]:
# Split data X and y into train and val sets
X_test = df_test.loc[:,['Pclass','Sex','Embarked','Age_binned','Family_size','Is_Alone']]
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)

In [258]:
#sc = StnadardScaler

#X = sc.fit_transform(X)

## Step 5: Train (or fit) the model

In [259]:
# Fit logistic regression
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Step 6: Evaluate the model

In [260]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [261]:
# Use model to predict validation set
y_pred_valid = LR.predict(X_valid)

In [262]:
print('Validation set accuracy', acc(y_valid, y_pred_valid), '%')

Validation set accuracy 0.8044692737430168 %


## Step 7: Generate Submission

In [263]:
# Predict for test set

y_pred_test = LR.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('basic_submission.csv', index=False)